In [ ]:
# 1. Seed setzen für Reproduzierbarkeit
import random, numpy as np, torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 2. Notwendige Bibliotheken importieren
import os
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForWholeWordMask,
    TrainingArguments,
    Trainer
)
# Für Fine-Tuning Evaluation (NER), falls benötigt:
# from seqeval.metrics import classification_report, f1_score

# 3. Pfad zur Textdatei (bitte anpassen)
file_path = "bachelorarbeit-ner/experiments/Experiment 4 - Domain Adaptive Pretraining/18thCenturyEnglish.txt"

# 4. Textdatei laden und in Absätze unterteilen (Absätze statt Einzelsätze)
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Annahme: Absätze sind durch doppelte Zeilenumbrüche getrennt
paragraphs = [p.strip() for p in text.split("\n\n") if p.strip() != ""]

# Dataset erstellen
dataset = Dataset.from_dict({"text": paragraphs})

# 5. Aufteilen in Trainings- und Validierungsdaten (95% / 5%)
raw_split = dataset.train_test_split(test_size=0.05, seed=42)
train_raw = raw_split["train"]
valid_raw = raw_split["test"]

# 6. Tokenizer laden (hier: RoBERTa Base)
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

# 7. Tokenisierungs-Funktion definieren (mit Truncation)
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        add_special_tokens=True,
        truncation=True,      # Aktiviert Truncation
        max_length=512,       # Maximale Tokenlänge festlegen
        return_attention_mask=True,
    )

# 8. Tokenisierung auf Trainings- und Validierungsdaten anwenden
tokenized_train = train_raw.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_valid = valid_raw.map(tokenize_function, batched=True, remove_columns=["text"])

# 9. Da wir Truncation nutzen, ist ein zusätzliches Gruppieren in Blöcke von 512 Tokens nicht mehr nötig.
# Daher verwenden wir die tokenisierten Datensätze direkt:
train_dataset = tokenized_train
valid_dataset = tokenized_valid

# 10. Data Collator für Whole Word Masking
data_collator = DataCollatorForWholeWordMask(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# 11. Modell laden 
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

# 12. TrainingArguments konfigurieren (Pretraining ausdehnen: mehr Epochen)
training_args = TrainingArguments(
    output_dir="bert_earlyamerican",
    overwrite_output_dir=True,
    num_train_epochs=6,                         # Mehr Epochen für längeres Pretraining
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100,
    logging_dir='./logs',                        # Logging-Verzeichnis für TensorBoard
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    lr_scheduler_type="linear",
    fp16=True,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# 14. Trainer erstellen 
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

# 15. Training starten
trainer.train()

